In [1]:
import warnings

warnings.simplefilter("ignore")

In [2]:
import mlflow

mlflow.set_experiment(experiment_name="heart-condition-classifier")

<Experiment: artifact_location='file:///C:/Users/SANDEEP/Downloads/cars_mlr%20%282%29/cars_mlr/mlruns/933766090133469887', creation_time=1671709492065, experiment_id='933766090133469887', last_update_time=1671709492065, lifecycle_stage='active', name='heart-condition-classifier', tags={}>

In [ ]:
#!pip install mlflow


In [3]:
import pandas as pd
file_url = r"Cars.csv"
df = pd.read_csv(file_url)

In [4]:
df

,MPG,Enginetype,HP,VOL,SP,WT
0,53.700681,petrol,49,89,104.185353,28.762059
1,50.013401,hybrid,55,92,105.461264,30.466833
2,50.013401,diesel,55,92,105.461264,30.193597
3,45.696322,lpg,70,92,113.461264,30.632114
4,50.504232,petrol,53,92,104.461264,29.889149
...,...,...,...,...,...,...
2494,43.469434,petrol,52,104,99.564907,34.483207
2495,35.404192,diesel,103,107,121.840817,35.549360
2496,39.431235,petrol,84,114,113.484609,37.042350
2497,39.431235,petrol,84,101,112.288996,33.234361


In [5]:
df.isnull().any()

MPG           False
Enginetype    False
HP            False
VOL           False
SP            False
WT            False
dtype: bool

In [6]:
X = df.iloc[:, 1:6].values
y = df.iloc[:, 0].values

In [7]:
df["Enginetype"].unique()

array(['petrol', 'hybrid', 'diesel', 'lpg', 'cng'], dtype=object)

In [8]:
mlflow.sklearn.autolog()

In [9]:
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer([("onehotencoder", OneHotEncoder(), [0])], remainder = "passthrough")
X = ct.fit_transform(X)
X

array([[0.0, 0.0, 0.0, ..., 89, 104.1853528, 28.7620589],
       [0.0, 0.0, 1.0, ..., 92, 105.4612635, 30.46683298],
       [0.0, 1.0, 0.0, ..., 92, 105.4612635, 30.19359657],
       ...,
       [0.0, 0.0, 0.0, ..., 114, 113.4846092, 37.04235003],
       [0.0, 0.0, 0.0, ..., 101, 112.2889958, 33.23436141],
       [1.0, 0.0, 0.0, ..., 97, 119.9211148, 31.38004084]], dtype=object)

In [10]:
run = mlflow.start_run()

2022/12/22 17:27:26 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh()

All git commands will error until this is rectified.

This initial warning can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|none|n|0: for no warning or exception
    - warn|w|warning|1: for a printed warning
    - error|e|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet



In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
from sklearn.linear_model import LinearRegression
multilinear = LinearRegression()
multilinear.fit(X_train,y_train)


LinearRegression()

In [12]:
X_test

array([[0.0, 1.0, 0.0, ..., 113, 143.3926389, 37.62069475],
       [0.0, 0.0, 0.0, ..., 111, 130.2086984, 36.88815313],
       [0.0, 1.0, 0.0, ..., 86, 127.909442, 28.07059654],
       ...,
       [0.0, 1.0, 0.0, ..., 91, 114.3692933, 29.92939368],
       [0.0, 0.0, 0.0, ..., 94, 115.6452041, 30.92015417],
       [0.0, 0.0, 0.0, ..., 115, 150.5765794, 37.92311321]], dtype=object)

In [13]:
y_pred = multilinear.predict(X_test)

In [ ]:
#y_pred

In [ ]:
#y_test

In [14]:
from sklearn.metrics import r2_score
accuracy = r2_score(y_test, y_pred)
accuracy

0.7339434097835325

In [15]:
mlflow.end_run()

In [16]:
run = mlflow.get_run(run.info.run_id)

In [17]:
pd.DataFrame(data=[run.data.params], index=["Value"]).T

,Value
copy_X,True
fit_intercept,True
n_jobs,None
positive,False


In [18]:
pd.DataFrame(data=[run.data.metrics], index=["Value"]).T

,Value
r2_score__,0.733943
training_mean_absolute_error,1.924514
training_mean_squared_error,8.022595
training_r2_score,0.781957
training_root_mean_squared_error,2.832419
training_score,0.781957


In [19]:
client = mlflow.tracking.MlflowClient()

In [20]:
client.list_artifacts(run_id=run.info.run_id)

[<FileInfo: file_size=4897, is_dir=False, path='estimator.html'>,
 <FileInfo: file_size=62, is_dir=False, path='metric_info.json'>,
 <FileInfo: file_size=None, is_dir=True, path='model'>]

In [27]:
classifier = mlflow.sklearn.load_model(f"runs:/{run.info.run_id}/model")

In [28]:
type(classifier)

sklearn.linear_model._base.LinearRegression

In [29]:
classifier.predict(X_test)

array([24.7401896 , 29.73670468, 33.83342495, 26.88950417, 32.27623404,
       35.24006476, 41.41179788, 35.24963766, 41.41179788, 36.28924846,
       23.69271507, 37.16623362, 24.7401896 , 24.74518525, 41.14680239,
       35.14640728, 29.79641341, 41.42538623, 29.79641341, 29.79641341,
       24.7401896 , 37.28905431, 35.83586534, 24.7401896 , 25.21044759,
       33.63847583, 37.38789562, 34.12103115, 31.30205421, 26.4663117 ,
       35.51569783, 35.3584534 , 24.7401896 , 37.16623362, 41.41179788,
       36.89824391, 34.37095676, 37.27532223, 43.79416167, 33.51615429,
       44.60593367, 38.02597297, 37.43876213, 37.77748663, 35.3584534 ,
       28.8323978 , 37.28905431, 36.1602144 , 33.51615429, 20.5150189 ,
       35.3584534 , 16.76515693, 36.15671195, 36.28924846, 36.89824391,
       41.42538623, 31.5025794 , 32.27623404, 37.0896145 , 25.75475141,
       31.30205421, 37.38789562, 37.43876213, 34.06590969, 26.4663117 ,
       37.77748663, 19.45217431, 36.15671195, 32.18928518, 34.37

In [30]:
# Reload the dataset
df = pd.read_csv(file_url)

In [31]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [32]:
pipeline = Pipeline(steps=[("model", classifier)])

In [33]:
pipeline

Pipeline(steps=[('model', LinearRegression())])

In [34]:
from mlflow.models import infer_signature

signature = infer_signature(X_test, y_test)

In [35]:
signature

inputs: 
  [Tensor('object', (-1, 9))]
outputs: 
  [Tensor('float64', (-1,))]

In [36]:
with mlflow.start_run() as run:
    pipeline.fit(X_train, y_train)
    mlflow.sklearn.log_model(pipeline, artifact_path="pipeline", signature=signature)

In [37]:
pipeline_model = mlflow.sklearn.load_model(f"runs:/{run.info.run_id}/pipeline")

In [38]:
type(pipeline_model)

sklearn.pipeline.Pipeline

In [39]:
pipeline_model.predict(X_test)

array([24.7401896 , 29.73670468, 33.83342495, 26.88950417, 32.27623404,
       35.24006476, 41.41179788, 35.24963766, 41.41179788, 36.28924846,
       23.69271507, 37.16623362, 24.7401896 , 24.74518525, 41.14680239,
       35.14640728, 29.79641341, 41.42538623, 29.79641341, 29.79641341,
       24.7401896 , 37.28905431, 35.83586534, 24.7401896 , 25.21044759,
       33.63847583, 37.38789562, 34.12103115, 31.30205421, 26.4663117 ,
       35.51569783, 35.3584534 , 24.7401896 , 37.16623362, 41.41179788,
       36.89824391, 34.37095676, 37.27532223, 43.79416167, 33.51615429,
       44.60593367, 38.02597297, 37.43876213, 37.77748663, 35.3584534 ,
       28.8323978 , 37.28905431, 36.1602144 , 33.51615429, 20.5150189 ,
       35.3584534 , 16.76515693, 36.15671195, 36.28924846, 36.89824391,
       41.42538623, 31.5025794 , 32.27623404, 37.0896145 , 25.75475141,
       31.30205421, 37.38789562, 37.43876213, 34.06590969, 26.4663117 ,
       37.77748663, 19.45217431, 36.15671195, 32.18928518, 34.37